In [32]:
import pandas as pd
import time 
import csv

In [112]:
year = 2014
month = 1
day = 28
path = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_'
best_price_number = 5

In [113]:
## read file
def read_file(year,month,day,path):
    data = []
    if len(str(month)) == 1:
        month_ = '0'+str(month)
    else:
        month_ = str(month)
    if len(str(day)) == 1:
        day_ = '0'+str(day)
    else:
        day_ = str(day)
    datapath = str(path) + str(year)+'.'+str(month_)+'.'+str(day_)+'.csv'
    data = pd.read_csv(datapath)
    data_CN14 = data[data.Series == data['Series'][0]]
    data = data_CN14
    return data

def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

def draw_out(order_book_data,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position+1:]
    return pd.concat((top,bottom)).reset_index(drop = True)

def order_book_to_csv(order_book_bid,order_book_ask,data,i):
    order_book_bid_sum = order_book_bid[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
    order_book_ask_sum = order_book_ask[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
    order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)    
    #print order_book_bid_ask
    with open('order_book_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv','a') as f:
        order_book = csv.writer(f)
        order_book.writerow(["TimeStamp",data.TimeStamp[i-1:i].iloc[0]])
        order_book = csv.writer(f,delimiter=',')
        for i in range(0,min(len(order_book_bid_ask),best_price_number),1):#len(order_book_bid_ask)):
            order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])
    return order_book_bid_sum,order_book_ask_sum

In [114]:
data = read_file(year,month,day,path)

In [115]:
with open('order_book_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv', 'wb') as csvfile:
    f = csv.writer(csvfile) 

In [135]:
data['QuantityDifference_'] = data['QuantityDifference']
data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
#order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
#order_book_bid = a[a.QuantityDifference != 0].reset_index(drop = True)
order_book_ask = b[b.QuantityDifference != 0].reset_index(drop = True)
order_book_bid.append(0)
#order_book_ask.append(0)
order_book_bid_sum = order_book_bid[0][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
order_book_ask_sum = order_book_ask[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [136]:
with open('order_book_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv','a') as f:
    order_book = csv.writer(f)
    order_book.writerow(["Bid","Bid_Quantity","Ask","Ask_Quantity"])
    order_book.writerow(["TimeStamp",timestamp])
    order_book = csv.writer(f,delimiter=',')
    for i in range(0,min(len(order_book_bid_ask),best_price_number),1):
        order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])

In [118]:
order_book_bid[0]

,Price,OrderNumber,QuantityDifference,QuantityDifference_
0,662000,2D8DE96295907A90,50,50


In [119]:
order_book_ask

,Price,OrderNumber,QuantityDifference,QuantityDifference_
0,689500,9672009CE4FAC9FA,2,2


In [49]:
order_book_bid

[    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  662000  2D8DE96295907A90                  50                   50,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  665000  0D86E172E8CCCB2C                   1                    1
 1  662000  2D8DE96295907A90                  50                   50,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  665000  0D86E172E8CCCB2C                   1                    1
 1  662500  A134C16B1EAE00EE                   1                    1
 2  662000  2D8DE96295907A90                  50                   50,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  665000  0D86E172E8CCCB2C                   1                    1
 1  662500  A134C16B1EAE00EE                   1                    1
 2  662000  2D8DE96295907A90                  50                   50
 3  653000  8D86E17A0D80EFE1                   1                    1,
     Price      

In [137]:
start  =  time.time()

for i in range(len(y),5000,1):#len(data),1):
    print i#,temp_bid,temp_ask
    time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
    if time_second > 57600:
        break
    if time_second == 32400 and time_second >= 57300:
        order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False]).reset_index(drop = True)
        #order_book_bid = order_book_bid.sort(['Price'],ascending = [False]).reset_index(drop = True)
        #order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True]).reset_index(drop = True)
        order_book_ask = order_book_ask.sort(['Price'],ascending = [True]).reset_index(drop = True)
        pass 
    
    if data.BidOrAsk[i] == 'A':
        data_ask_Quantity = data.BestQuantity[i]
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0] and time_second < 32400:
            #if order_book_bid.Price >= data[i:i+1].Price.iloc[0] and time_second < 32400:
                for k in range(0,len(order_book_bid[temp_bid])):
                    diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = -diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                print order_book_ask
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                #order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                order_book_ask = insert(order_book_ask,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                #length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                if time_second > 32400 and time_second < 57300: 
                    #if position_ == 0 and len(order_book_ask[temp_ask]) > 1
                    if position_ == 0 and len(order_book_ask) > 1:   
                        if order_book_ask[position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:  
                            print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            pass
                    #elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    elif 0 < position_< (len(order_book_ask)-1):
                        #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        if order_book_ask[position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'   
                            pass
                    #elif position_ == len(order_book_ask[temp_ask])-1:
                    elif position_ == len(order_book_ask)-1:    
                        #if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        if order_book_ask[position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:        
                            print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'                
                            pass
                    #elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                    elif position_ == 0 and len(order_book_ask) == 1:
                        #print 'position and Best Price is True' 
                        pass
                else:
                    #print 'before open market'
                    pass
            elif data.TimeStamp[i] != x1[temp_ask]:
                if temp_ask == 0:
                    print 'timestamp = %s'%(data.TimeStamp[i])
                    print order_book_ask
                    
                    temp_ask = temp_ask + 1
                    #order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    #order_book_ask = order_book_ask
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    #order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                    order_book_ask = insert(order_book_ask,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                    
                    if time_second > 32400 and time_second < 57300:
                        if position_ == 0 and len(order_book_ask[0]) > 1:
                            #if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                                print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass
                        #elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        elif 0 < position_< len(order_book_ask)-1:
                            #if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            if order_book_ask[position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        #elif position_ == len(order_book_ask[temp_ask])-1:  
                        elif position_ == len(order_book_ask)-1: 
                            #if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                #print 'position and Best Price is True'
                                pass
                        #elif position_ == 0 and len(order_book_ask[0]) == 1:
                        elif position_ == 0 and len(order_book_ask) == 1:
                            #print 'position and Best Price is True'
                            pass
                    else:
                        #print 'before open market'
                        pass
                else:
                    print 'timestamp = %s'%(data.TimeStamp[i])
                    print order_book_ask
                    
                    #order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask,data,i)
                    if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                    round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                        if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                            if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                #print 'Best ask quantity is ture'
                                pass
                            else:
                                print 'Best ask quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                    if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        #print 'Best ask quantity is ture'
                                        break
                                else:
                                    j = j - 1
                                    pass
                    else:
                        pass
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    #order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    #order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                    order_book_ask = insert(order_book_ask,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                    if time_second > 32400 and time_second < 57300:
                        if position_ == 0:
                            #if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass
                        #elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        elif 0 < position_< len(order_book_ask)-1:
                            #if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        #elif position_ == len(order_book_ask[temp_ask])-1: 
                        elif position_ == len(order_book_ask)-1: 
                            #if order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                #print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            #print 'position and Best Price is True'
                            pass
                    else:
                        #print 'before open market'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x1[temp_ask]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                print order_book_ask
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                #position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                position_ = order_book_ask[order_book_ask.OrderNumber == order_number_].index[0]                
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400 and time_second < 57300:
                    #if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                    if position_ == 0 and len(order_book_ask) > 1:
                        #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:    
                            print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            pass       
                    #elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    elif 0 < position_< len(order_book_ask)-1:
                        #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:                             
                            print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'  
                            pass
                    #elif position_ == len(order_book_ask[temp_ask])-1:  
                    elif position_ == len(order_book_ask)-1:
                        #if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        if position_ > 0 and order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                            break
                        elif position_ == 0:
                            #print 'position and Best Price is True'
                            pass                        
                        else:
                            #print 'position and Best Price is True'                        
                            pass
                    #elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                    elif position_ == 0 and len(order_book_ask) == 1:    
                        #print 'position and Best Price is True'
                        pass
                else:
                    #print 'before open market' 
                    pass
                #if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                if order_book_ask[(order_book_ask.OrderNumber == order_number_)&(order_book_ask.Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):        
                    #order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                    order_book_ask = order_book_ask.drop(order_book_ask.index[[position_]]).reset_index(drop = True)                
                else:
                    #order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] = order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
            
            elif data.TimeStamp[i] != x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                print order_book_ask
                
                order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask,data,i)
                if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                    if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                        if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                            #print 'Best ask quantity is ture'
                            pass
                        else:
                            print 'Best ask quantity is false'
                            break 
                    else:
                        j = i - 1
                        while j >= 1:
                            #print j
                            #print data[j-1:j].BidOrAsk.iloc[0]
                            if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                    #print 'Best ask quantity is ture'
                                    break
                            else:
                                j = j - 1
                                pass
                #order_book_ask.append(order_book_ask[temp_ask])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                #position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                position_ = order_book_ask[order_book_ask.OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                if time_second > 32400 and time_second < 57300:
                    #if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                    if position_ == 0 and len(order_book_ask) > 1:
                        #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        if order_book_ask[position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            pass
                    #elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    elif 0 < position_< len(order_book_ask)-1:    
                        #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        if order_book_ask[position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:                             
                            print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            #print 'position and Best Price is True'  
                            pass
                    #elif position_ == len(order_book_ask[temp_ask])-1:
                    elif position_ == len(order_book_ask)-1:
                        #if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        if position_ > 0 and order_book_ask[position_-1:position_-1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            #print 'position and Best Price is True'
                            pass                       
                        else:
                            #print 'position and Best Price is True'                        
                            pass
                    #elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                    elif position_ == 0 and len(order_book_ask) == 1:
                        #print 'position and Best Price is True'
                        pass      
                else:
                    #print 'before open market'
                    pass
                #if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                if order_book_ask[(order_book_ask.OrderNumber == order_number_)&(order_book_ask.Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):                    
                    #order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                    order_book_ask = order_book_ask.drop(order_book_ask.index[[position_]]).reset_index(drop = True)
                else:
                    #order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] = order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    
    elif data.BidOrAsk[i] == 'B':
        data_bid_Quantity = data.BestQuantity[i]
        if int(data[['QuantityDifference']][i:i+1].values) > 0: 
            
            #if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0] and time_second < 32400:
            if order_book_ask.Price[0] <= data[i:i+1].Price.iloc[0] and time_second < 32400:
                #for k in range(0,len(order_book_ask[temp_ask])):
                for k in range(0,len(order_book_ask)):
                    #diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    diff = order_book_ask.QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    #if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                    if order_book_ask.Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                        #order_book_ask[temp_ask].QuantityDifference_[k] = diff
                        order_book_ask.QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    #elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                    elif order_book_ask.Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                        #order_book_ask[temp_ask].QuantityDifference_[k] = 0
                        order_book_ask.QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = - diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                if time_second > 32400 and time_second < 57300:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                
                            print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            pass     
                    elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break    
                        else:
                            #print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        #print 'position and Best Price is True'
                        pass 
                else:
                    #print 'before open market'
                    pass
            elif data.TimeStamp[i] != x2[temp_bid]:
                #print temp_bid
                print 'timestamp = %s'%(data.TimeStamp[i])
                if temp_bid == 0:
                    best_price = data[i:(i+1)]['BestPrice']             
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                 
                    if time_second > 32400 and time_second < 57300:
                        if position_ == 0  and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass       
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:    
                            if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                                pass
                            else:
                                #print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                                #print 'position and Best Price is True'
                                pass
                    else:
                        #print 'before open market'
                        pass
                else:
                    if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                    round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                        #order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)                    
                        order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask,data,i)                    
                        if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                            if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                #print 'Best bid quantity is ture'
                                pass
                            else:
                                print 'Best bid quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                #print j
                                #print data[j-1:j].BidOrAsk.iloc[0]
                                if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                    if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        #print 'Best bid quantity is ture'
                                        break
                                    else:
                                        print 'Best bid quantity is false'
                                        break
                                else:
                                    j = j - 1
                                    pass
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1 
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                    if time_second > 32400 and time_second < 57300:
                        if position_ == 0 and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                #print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                            #print 'position and Best Price is True'
                            pass
                    else:
                        #print 'before open market'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x2[temp_bid]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400 and time_second < 57300:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            pass               
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            #print 'position and Best Price is True'
                            pass
                        else:
                            #print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        #print 'position and Best Price is True'
                        pass
                else:
                    #print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
            elif data.TimeStamp[i] != x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                    #order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask,data,i)
                    if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                        if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                            #print 'Best bid quantity is ture'
                            pass
                        else:
                            print 'Best bid quantity is false'
                            break 
                    else:
                        j = i - 1
                        while j >= 1:
                            #print j
                            #print data[j-1:j].BidOrAsk.iloc[0]
                            if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                    #print 'Best bid quantity is ture'
                                    break
                                else:
                                    print 'Best bid quantity is false'
                                    break
                            else:
                                j = j - 1
                                pass 
                order_book_bid.append(order_book_bid[temp_bid])
                order_number_ =  data['OrderNumber'][i:i+1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                if time_second > 32400 and time_second < 57300:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            #print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        elif position_ == 0:
                            #print 'position and Best Price is True'
                            pass
                        else:
                            #print 'position and Best Price is True'                        
                            pass    
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        #print 'position and Best Price is True'
                        pass  
                else:
                    #print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
end = time.time()            
print "Total time = %f"%(end - start)

KeyboardInterrupt: 

In [131]:
data.head(30)

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
0,CNF14,1,2014-01-28D04:18:19.800793700,9672009CE4FAC9FA,1,689500,2,,A,689500,2,2
1,CNF14,2,2014-01-28D04:18:19.800793700,2D8DE96295907A90,1,662000,50,,B,662000,50,50
2,CNF14,3,2014-01-28D08:45:01.501087400,0D86E172E8CCCB2C,1,665000,1,,B,665000,1,1
3,CNF14,4,2014-01-28D08:45:01.584087400,A134C16B1EAE00EE,2,662500,1,,B,665000,1,1
4,CNF14,5,2014-01-28D08:45:02.060075400,8D86E17A0D80EFE1,4,653000,1,,B,665000,1,1
5,CNF14,6,2014-01-28D08:45:02.062075400,07526D6268794A99,5,653000,1,,B,665000,1,1
6,CNF14,7,2014-01-28D08:45:02.209075400,87526D62EC2FCE4F,4,653500,1,,B,665000,1,1
7,CNF14,8,2014-01-28D08:45:02.374075400,87D26D6223040524,2,663500,1,,B,665000,1,1
8,CNF14,9,2014-01-28D08:45:27.597775400,8B68CD232EC71107,2,689500,1,,A,689500,3,1
9,CNF14,10,2014-01-28D08:45:32.346715400,0BE0CD2BCC85AEC5,3,690000,1,,A,689500,3,1


In [138]:
order_book_bid[temp_bid]

,Price,OrderNumber,QuantityDifference,QuantityDifference_
0,671500,74B26A59FA73DD93,10,10
1,671500,D024CE4D8DA96CE9,10,10
2,671500,5E97038D46042684,10,10
3,671500,D077EA4999817941,6,6
4,671500,DEA2E241D183B0E3,10,10
5,671500,545F83C591C07240,10,10
6,671500,5E1F038D374E17CE,10,10
7,671500,50ACCE4D2AE00A20,6,10
8,671500,72FF6649EB7DCCBD,10,10
9,671500,DAF5E251518A354A,10,10


In [139]:
order_book_ask

,Price,OrderNumber,QuantityDifference,QuantityDifference_
0,672500,F2FEEE0116F9F5FA,20,20
1,672500,F46AEE11ED53D073,1,1
2,672500,D4BAEE09205EFF5F,20,20
3,672500,F6FD0ED94D032DA3,10,10
4,672500,FA2602C929B10A51,10,10
5,672500,F6758ED10C6CED0D,10,10
6,672500,F4FA0EC9363A16DA,10,10
7,672500,F0FF6E11C7AEA8EE,3,3
8,672500,52A3E641810860C8,10,10
9,672500,D03B6659B52F94EF,10,10


In [130]:
order_book_ask

,Price,OrderNumber,QuantityDifference,QuantityDifference_
0,668500,5CFA6619E18DC4AD,45,45
1,675000,FC7AC2555EDE431E,65,65
2,675500,DCF26619B2179537,65,65
3,676000,5C7A661977E95B09,130,130
4,676500,7CFAC2555DD14211,130,130
5,677000,FCFAC2550E40F281,192,192
6,678000,DC7A661981EE650E,1,1
7,680000,D4FA6E113B361E56,2,2
8,686000,DCFAE61989816CA1,1,1
9,689500,9672009CE4FAC9FA,2,2


In [129]:
pd.read_table('order_book_2014_1_28.csv',sep=',')[50:120]

,Bid,Bid_Quantity,Ask,Ask_Quantity
50,TimeStamp,2014-01-28D08:46:00.073379400,NaN,NaN
51,666000.0,65.0,689500.0,3.0
52,665500.0,65.0,691000.0,20.0
53,665000.0,131.0,697000.0,27.0
54,664500.0,130.0,NaN,NaN
55,664000.0,192.0,NaN,NaN
56,TimeStamp,2014-01-28D08:46:00.075379400,NaN,NaN
57,666000.0,65.0,675500.0,65.0
58,665500.0,65.0,676500.0,130.0
59,665000.0,131.0,689500.0,3.0


In [127]:
data.head(40)

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
0,CNF14,1,2014-01-28D04:18:19.800793700,9672009CE4FAC9FA,1,689500,2,,A,689500,2,2
1,CNF14,2,2014-01-28D04:18:19.800793700,2D8DE96295907A90,1,662000,50,,B,662000,50,50
2,CNF14,3,2014-01-28D08:45:01.501087400,0D86E172E8CCCB2C,1,665000,1,,B,665000,1,1
3,CNF14,4,2014-01-28D08:45:01.584087400,A134C16B1EAE00EE,2,662500,1,,B,665000,1,1
4,CNF14,5,2014-01-28D08:45:02.060075400,8D86E17A0D80EFE1,4,653000,1,,B,665000,1,1
5,CNF14,6,2014-01-28D08:45:02.062075400,07526D6268794A99,5,653000,1,,B,665000,1,1
6,CNF14,7,2014-01-28D08:45:02.209075400,87526D62EC2FCE4F,4,653500,1,,B,665000,1,1
7,CNF14,8,2014-01-28D08:45:02.374075400,87D26D6223040524,2,663500,1,,B,665000,1,1
8,CNF14,9,2014-01-28D08:45:27.597775400,8B68CD232EC71107,2,689500,1,,A,689500,3,1
9,CNF14,10,2014-01-28D08:45:32.346715400,0BE0CD2BCC85AEC5,3,690000,1,,A,689500,3,1
